# set SYMPHONY_API_KEY as an environment variable before importing pysymphony

In [2]:
from pysymphony import SymphonyClient, settings
import logging

# set logging level to INFO for basic logs
logging.basicConfig(level=logging.INFO)

# add a local tool
# ensure to add a docstring for the tool description, and define the input parameter types
def add_tool(a: int, b: int):
    """
    Add two numbers
    """
    print(f"Adding {a} and {b}")
    return a + b

settings.add_local_tool(add_tool)

INFO:root:GET https://inference.composition-labs.com/inference/tools/list: RUNNING
INFO:root:GET https://inference.composition-labs.com/inference/tools/list: SUCCESS


In [3]:
# initialize the symphony client
client = SymphonyClient()

In [4]:
# check available tools
client.tools

[{'id': 'pplx_online',
  'name': 'Perplexity Online',
  'description': 'Useful for getting internet enabled (access to internet) natural language answers to input queries without citations.',
  'type': <ToolType.SYMPHONY: 'symphony'>},
 {'id': 'openai_img_to_text',
  'name': 'OpenAI Image to Text',
  'description': 'Useful for analyzing and describing images, detecting objects, and extracting text from images.',
  'type': <ToolType.SYMPHONY: 'symphony'>},
 {'id': 'clinical_trials_gov_data_retriever',
  'name': 'Clinical Trials Gov Data Retriever',
  'description': 'Useful for retrieving a list of clinical studies for the pharma and biotech indistries, based on a given query, or details of a single clinical study based on a given NCT number. Pass studies_query to get a list of studies based on the query. Pass nct_id to get details of a single study. Never pass both studies_query and nct_id.',
  'type': <ToolType.SYMPHONY: 'symphony'>},
 {'id': 'competitor_analysis_tool',
  'name': 'Comp

In [5]:
# generate a workflow

# 1. define relevant parameters
useful_tools = ["add_tool"]
parameters = {
    "max_steps": 1
}
task_description = "For the given list of numbers, add them all up"

# 2. generate the workflow
workflow = client.generate_workflow(task_description=task_description, tools=useful_tools, parameters=parameters)

# print the workflow
print(workflow.__dict__)

INFO:root:POST https://inference.composition-labs.com/inference/planner/generateworkflow: SUCCESS
INFO:root:GET https://inference.composition-labs.com/inference/planner/getworkflows: RUNNING
INFO:root:GET https://inference.composition-labs.com/inference/planner/getworkflows: SUCCESS


{'id': '7cf8b1bc-1dba-47ad-9e0f-badb22ba3c87', 'task_description': 'For the given list of numbers, add them all up', 'nodes': [{'id': '0ef73128-c8cf-48bd-8fc0-7af2f7d96e63', 'step_index': 1, 'input': 'A list of numbers', 'job': 'Sum all the numbers in the given list to produce a single total value.', 'output': 'The total sum of the numbers in the list', 'prompt': 'Given a list of numbers as input, use the add_tool to iteratively add each number in the list, ensuring that you accurately calculate and return a single numerical output representing the total sum of all the numbers in the list.', 'dependencies': [], 'created_at': '2024-10-08T02:04:17.883360', 'updated_at': '2024-10-08T02:04:17.883360', 'tools': [{'id': '1b8562a8-cda5-4514-929d-b819a30ab72e', 'unique_id': 'add_tool', 'name': 'add_tool', 'description': 'Add two numbers', 'tool_type': 'client', 'schema': {'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'ti

In [6]:
# run the workflow

# 1. initialize the run
run = workflow.init_run(input="12313123, 12312312324, 12313, 123123123")

INFO:root:POST https://inference.composition-labs.com/inference/runner/initrun: SUCCESS


In [7]:
run.run()

INFO:root:POST https://inference.composition-labs.com/inference/runner/runstep: SUCCESS


Adding 12313123 and 12312312324


INFO:root:POST https://inference.composition-labs.com/inference/runner/runstep: SUCCESS


Adding 12313123 and 12312312324
Adding 12324625447 and 12313


INFO:root:POST https://inference.composition-labs.com/inference/runner/runstep: SUCCESS


Adding 12313123 and 12312312324
Adding 12324625447 and 12313
Adding 12324637760 and 123123123


INFO:root:POST https://inference.composition-labs.com/inference/runner/runstep: SUCCESS


<Status.SUCCESS: 'success'>

In [8]:
run.ios

{'0': {'input': None, 'output': '12313123, 12312312324, 12313, 123123123'},
 '1': {'input': 'A list of numbers: 12313123, 12312312324, 12313, 123123123',
  'output': 'The total sum of the numbers in the list is 12,447,760,883.'}}